In [47]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_data_40k.csv')
df = df[:1000]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_demo.csv',index=False)


In [48]:
df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_data_5k.csv')
df = df[:125]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_demo.csv',index=False)

In [38]:
import yaml
from easydict import EasyDict

with open('/hpc2hdd/home/yli106/smiles2mol/config/qm9_default.yml', 'r') as file:
    config = yaml.safe_load(file)

config = EasyDict(config)

In [42]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import argparse
from easydict import EasyDict

tokenizer = AutoTokenizer.from_pretrained("/hpc2hdd/home/yli106/smiles_3d/llama2_7b_chat", trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token # Use the EOS token to pad shorter sequences
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

/hpc2hdd/home/yli106/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/hpc2hdd/home/yli106/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [59]:
from tqdm import tqdm

max_len = 0
for i in tqdm(range(len(df))):
    input_text = df['text'][i]
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    if len(input_ids[0]) > max_len:
        max_len = len(input_ids[0])
max_len   

100%|██████████| 200000/200000 [10:49<00:00, 307.90it/s]


1881

In [60]:
df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_demo.csv')
df

,smiles,canonicalize_smiles,num_atom,num_bond,mol_block,text
0,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
1,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
2,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
3,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
4,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
...,...,...,...,...,...,...
995,COC[C@H]1CO[C@H]1CO,[H]OC([H])([H])[C@]1([H])OC([H])([H])[C@]1([H]...,21,21,\n RDKit 3D\n\n 21 21 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
996,COC[C@H]1CO[C@H]1CO,[H]OC([H])([H])[C@]1([H])OC([H])([H])[C@]1([H]...,21,21,\n RDKit 3D\n\n 21 21 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
997,COC[C@H]1CO[C@H]1CO,[H]OC([H])([H])[C@]1([H])OC([H])([H])[C@]1([H]...,21,21,\n RDKit 3D\n\n 21 21 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
998,COC[C@H]1CO[C@H]1CO,[H]OC([H])([H])[C@]1([H])OC([H])([H])[C@]1([H]...,21,21,\n RDKit 3D\n\n 21 21 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...


In [61]:
print(df['text'][0])

<s>[INST] <<SYS>>
Below is a SMILES of a molecule, generate its 3D structure. The molecule has 19 atoms and 20 bonds.
<</SYS>>

[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1([H])[H])C([H])([H])[H] [/INST] 
     RDKit          3D

 19 20  0  0  0  0  0  0  0  0999 V2000
   -0.6071   -1.7891    0.0790 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.1418   -0.3532   -0.0397 C   0  0  2  0  0  0  0  0  0  0  0  0
    1.1536   -0.1939    0.5447 N   0  0  0  0  0  0  0  0  0  0  0  0
    1.9413    0.9960    0.3724 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.3320   -0.2693   -0.2849 C   0  0  0  0  0  0  0  0  0  0  0  0
    3.1456   -0.8561    0.1180 H   0  0  0  0  0  0  0  0  0  0  0  0
    2.2045   -0.3448   -1.3597 H   0  0  0  0  0  0  0  0  0  0  0  0
    2.4688    1.3178    1.2606 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.5741    1.8084   -0.2445 H   0  0  0  0  0  0  0  0  0  0  0  0
   -1.1571    0.7476    0.1205 C   0  0  1  0  0  0  0  0  0  0  0  0
   -0.6135    0.4323   -1.244